In [1]:
print("https://www.reddit.com/r/orioles/search?sort=new&restrict_sr=on&q=flair:Lucky%2BGuess")

https://www.reddit.com/r/orioles/search?sort=new&restrict_sr=on&q=flair:Lucky%2BGuess


# Get the latest scores

In [2]:
import statsapi
from datetime import date, timedelta

for x in range(4):
    today = (date.today() - timedelta(days=x)).strftime("%Y-%m-%d")
    team_id = 110
    schedule_data = statsapi.schedule(date=today, team=team_id)
    
    if schedule_data:
            for game in schedule_data:
                print(f"Game ID: {game['game_id']}")
                print(f"Date: {game['game_date']}",f"{game['home_name']} ({game['home_score']})", "::",f"{game['away_name']} ({game['away_score']})")
                print("-" * 20)
    else:
        print(f"No games found for team ID {team_id} on {today}.")
        print("-" * 20)

Game ID: 776579
Date: 2025-08-27 Baltimore Orioles (0) :: Boston Red Sox (0)
--------------------
Game ID: 776591
Date: 2025-08-26 Baltimore Orioles (0) :: Boston Red Sox (5)
--------------------
Game ID: 776604
Date: 2025-08-25 Baltimore Orioles (3) :: Boston Red Sox (4)
--------------------
Game ID: 776612
Date: 2025-08-24 Baltimore Orioles (3) :: Houston Astros (2)
--------------------


In [3]:
s_id = '1n0a5zg'
the_score = [5,0]
target_team = "Orioles"
winning_team = "Red Sox"
publish_entries = True

run_today = True #Are we running it for today (True) or tomorrow (False)
postponed = False # Did we have a postponed game
days_to_skip = 1 # 1 to run for tomorrow add more if multiple days off

# The ID of the spreadsheet.
SPREADSHEET_ID = '1TxZmOdTsMRsLlpJQ5Bu04e3-kWuOdRvK40L4eRYYmWU'

In [4]:
import praw
import random
import os.path

#importing local .py files that have lookups and stuff
import team_lookup # Used to pull in the team subreddits so i dont have to type them out
import tie_breakers # This is a list of tie breakers that will randomly populate in the template
import config # this is where I store the PRAW credentials

import pandas as pd
import numpy as np

#Used to auto create some text
import google.generativeai as genai

# Used to make permalinks clickable in tables
from IPython.display import display, HTML, Markdown

from io import StringIO
from datetime import datetime, timedelta

from urllib.parse import quote

#Required for Google Sheets Uploads
import google.auth
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [5]:
# Function for creating Pandas Dataframe links for validation if needed
def make_clickable(url, name):
    return '<a href="{}" rel="noopener noreferrer" target="_blank">{}</a>'.format(url,name)

# Used to see if a value was already added to the plus_six list
def check_value_in_list_of_lists(list_of_lists, value):
    for sublist in list_of_lists:
        if value in sublist:
            return True
    return False

def get_scores(scores):
    tmp = scores.split('-')
    
def parse_score(comment):
    try:
        #print comment
        data = comment.split(' ')
        score = data[0].split('-')
        team = data[1].replace(',','')
        
        score[0] = int(score[0])
        score[1] = int(score[1])
        
        score.append(team)
        
        if len(data) > 2:
            tie_break = data[2]
            score.append(tie_break)
        else:
            #print data, 'incorrect input'
            incorrect_data.append(comment)
        
        return score
    except:
        print('[X] - Failed on: ' + comment)
        return

def append_values(spreadsheet_id, range_name, value_input_option, _values):
  """
  Creates the batch_update the user has access to.
  Load pre-authorized user credentials from the environment.
  TODO(developer) - See https://developers.google.com/identity
  for guides on implementing OAuth2 for the application.
  """

  try:
    service = build("sheets", "v4", credentials=creds)

    values = [
        [
            # Cell values ...
        ],
        # Additional rows ...
    ]
    body = {"values": _values}
    result = (
        service.spreadsheets()
        .values()
        .append(
            spreadsheetId=spreadsheet_id,
            range=range_name,
            valueInputOption=value_input_option,
            body=body,
        )
        .execute()
    )
    print(f"{(result.get('updates').get('updatedCells'))} cells appended.")
    return result

  except HttpError as error:
    print(f"An error occurred: {error}")
    return error


In [6]:
if not run_today:
    today = datetime.now() + timedelta(days=days_to_skip)
else:
    today = datetime.now()

year = today.strftime("%Y")
month = today.strftime("%m")
day = today.strftime("%d")

r = praw.Reddit(
    client_id = config.client_id,
    client_secret = config.client_secret,
    user_agent = config.user_agent,
)

submission = r.submission(s_id)

# This is used to establish the "range" for +/- 1 results
top_score = []
bottom_score = []

incorrect_data = []
review_data = []
plus_six = []
plus_one = []
entries = []

top_score.append(the_score[0]-1)
top_score.append(the_score[0])
top_score.append(the_score[0]+1)
    
bottom_score.append(the_score[1]-1)
bottom_score.append(the_score[1])
bottom_score.append(the_score[1]+1)



In [7]:
print('pasing exact...')
for x in submission.comments:
    #Create permalink for submission review
    permalink = make_clickable('https://reddit.com' + x.permalink, 'Link')
    sheets_permalink = "=HYPERLINK(\"https://reddit.com" + x.permalink + "\", \"Link\")"
    
    try:
        tmp = parse_score(x.body)
    except:
        incorrect_data.append(x)

    if tmp is None:
        continue
    
    if len(tmp) < 4:
        incorrect_data.append(x)
        continue
    
    # Establish the list for the google sheets upload
    date = submission.title.split(' ')[2]
    entries.append([date,
                '/u/' + str(x.author),
                str(tmp[0]) + '-' + str(tmp[1]),
                tmp[2],
                tmp[3], 
                datetime.fromtimestamp(x.created_utc).strftime('%Y-%m-%d %H:%M:%S'), 
                sheets_permalink,
                'False'])
        
    if tmp[0] == the_score[0] and tmp[1] == the_score[1]:
        if tmp[2] in team_lookup.team_selection[winning_team]: # Check if the team is correct
            # See if the submission is an edited post to flag it
            if x.edited:
                plus_six.append(['e_/u/' + str(x.author),#.ljust(25,' '), 
                                 tmp[2],
                                 tmp[3], 
                                 datetime.fromtimestamp(x.edited).strftime('%Y-%m-%d %H:%M:%S'), 
                                 permalink, 
                                 'True'])
            else:
                plus_six.append(['/u/' + str(x.author),#.ljust(25,' '), 
                                 tmp[2],
                                 tmp[3], 
                                 datetime.fromtimestamp(x.created_utc).strftime('%Y-%m-%d %H:%M:%S'), 
                                 permalink,
                                 'False'])
        else:
            review_data.append(['/u/' + str(x.author),#.ljust(25,' '),
                                 tmp[2],
                                 tmp[3], 
                                 datetime.fromtimestamp(x.edited).strftime('%Y-%m-%d %H:%M:%S'), 
                                 permalink, 
                                 'True'])
print('parsing +/- 1...\n')
for x in submission.comments:
    permalink = make_clickable('https://reddit.com' + x.permalink, 'Link')
    try:
        tmp = parse_score(x.body)
    except:
        incorrect_data.append(x)

    if tmp is None:
        continue
    
    if len(tmp) < 4:
        incorrect_data.append(x)
        continue
    
    # Check if they were already recorded for plus_six exclude if they were
    if tmp[0] in top_score and tmp[1] in bottom_score and not check_value_in_list_of_lists(plus_six, '/u/' + str(x.author)):
        if tmp[2] in team_lookup.team_selection[winning_team]: # Check if the team is correct
            if x.edited:
                plus_one.append(['/u/' + str(x.author),#.ljust(25,' '),
                                 tmp[2],
                                 tmp[3], 
                                 datetime.fromtimestamp(x.created_utc).strftime('%Y-%m-%d %H:%M:%S'), 
                                 permalink, 
                                 'True'])
            else:
                plus_one.append(['/u/' + str(x.author),#.ljust(25,' '), 
                                 tmp[2],
                                 tmp[3], 
                                 datetime.fromtimestamp(x.created_utc).strftime('%Y-%m-%d %H:%M:%S'), 
                                 permalink, 
                                 'False'])
        else:
            review_data.append(['/u/' + str(x.author),#.ljust(25,' '),
                                 tmp[2],
                                 tmp[3], 
                                 datetime.fromtimestamp(x.created_utc).strftime('%Y-%m-%d %H:%M:%S'), 
                                 permalink, 
                                 'True'])

pasing exact...
parsing +/- 1...



# Plus Six Points Group

In [8]:
df_plus_six = pd.DataFrame(columns=list(['user', 'team', 'tie_break', 'post_date', 'Link', 'IsEdited']))

for x in plus_six:
    df_plus_six.loc[len(df_plus_six)] = x

display(HTML(df_plus_six.to_html(escape=False)))

,user,team,tie_break,post_date,Link,IsEdited


# PLUS One Point Group

In [9]:
df_plus_one = pd.DataFrame(columns=list(['user', 'team', 'tie_break', 'post_date', 'Link', 'IsEdited']))

for x in plus_one:
    #print('\t\t'.join(x))
    df_plus_one.loc[len(df_plus_one)] = x

display(HTML(df_plus_one.to_html(escape=False)))

,user,team,tie_break,post_date,Link,IsEdited


## Review Data

In [10]:
df_review = pd.DataFrame(columns=list(['user', 'team', 'tie_break', 'post_date', 'Link', 'IsEdited']))

for x in review_data:
    #print('\t\t'.join(x))
    df_review.loc[len(df_review)] = x

display(HTML(df_review.to_html(escape=False)))

,user,team,tie_break,post_date,Link,IsEdited


In [11]:
# These are the folks that dont know how to follow rules, include at your discretion :D
#incorrect = dict.fromkeys(incorrect_data).keys()

for x in incorrect_data:
    try:
        print(x.author, x.body)
    except AttributeError:
        print('\t'.join([x[0], x[1], x[2]]))
    except:
        print(x)

## Update the scoreboard

In [12]:
#df_table_test = pd.read_html(StringIO(submission.selftext_html))

In [13]:
#df_table_test[0]

In [14]:
#df_table_test[0]['Exact'] = np.where(df_table[0].Username == x[0], df_table[0]['Exact'] + 1, df_table[0]['Exact'])

In [15]:
f = StringIO(submission.selftext_html)
df_table = pd.read_html(StringIO(submission.selftext_html))

for x in plus_one:
    value_to_check = x[0]
    check = df_table[0]['Username'].isin([value_to_check])
    
    if check.any(): #Is the username already listed...if so update its values
        df_table[0]['+/-1'] = np.where(df_table[0].Username == x[0], df_table[0]['+/-1'] + 1, df_table[0]['+/-1'])
    else: # Create a new player and set +/-1 to 1 since this is their first rodeo
        new_entry = [0, x[0], 0, 1, 0, 0]
        df_table[0].loc[len(df_table[0])] = new_entry
        

for x in plus_six:
    value_to_check = x[0]
    check = df_table[0]['Username'].isin([value_to_check])
    
    if check.any(): #Is the username already listed...if so update its values
        df_table[0]['Exact'] = np.where(df_table[0].Username == x[0], df_table[0]['Exact'] + 1, df_table[0]['Exact'])
    else: # Create a new player and set exact to 1 since this is their first rodeo 
        new_entry = [0, x[0], 1, 0, 0, 0]
        df_table[0].loc[len(df_table[0])] = new_entry

# Calculate the Scores
df_table[0]['Total Points'] =  (df_table[0]['Exact'] * 6) + (df_table[0]["+/-1"] * 1) + (df_table[0]["Lost Tie"] * 2)

# Set the Rank
df_table[0].sort_values(by='Total Points', ascending=False, inplace=True)
df_table[0]['Rank'] = df_table[0].reset_index(drop=True).index + 1

print(df_table[0].sort_values(by='Total Points', ascending=False).to_markdown(index=False))
scoreboard = df_table[0].sort_values(by='Total Points', ascending=False).to_markdown(index=False)

|   Rank | Username               |   Exact |   +/-1 |   Lost Tie |   Total Points |
|-------:|:-----------------------|--------:|-------:|-----------:|---------------:|
|      1 | /u/KiwisOfWrath        |       2 |      2 |          0 |             14 |
|      2 | /u/zombiereign         |       1 |      1 |          0 |              7 |
|      3 | /u/malkusm             |       0 |      7 |          0 |              7 |
|      4 | /u/kingfiasco          |       0 |      6 |          0 |              6 |
|      5 | /u/bromanow025         |       1 |      0 |          0 |              6 |
|      6 | /u/jjk2                |       0 |      4 |          0 |              4 |
|      7 | /u/ruffyen             |       0 |      3 |          0 |              3 |
|      8 | /u/Birdland-Flock      |       0 |      3 |          0 |              3 |
|      9 | /u/Jeff_Banks_Monkey   |       0 |      2 |          0 |              2 |
|     10 | /u/Froslitz            |       0 |      2 |          0

## Create Congratulatory AI Text

In [16]:
congrats_text = "UPDATE THIS!!!!!!!!!!!!!!!!!!!!!!"

In [17]:
print(len(df_plus_six['user'].to_list()))
print(len(df_plus_one['user'].to_list()))

0
0


In [18]:
genai.configure(api_key=config.ai_api_key)
model = genai.GenerativeModel('gemini-2.0-flash')

def generate_output(prompt):
    response = model.generate_content(
        prompt,
        request_options = {
            "timeout": 1000
        }
    )

    try:
        return response.text
    except Exception as ex:
        raise ex

print("Number of Exact: {}\nNumber of +/-1: {}\n".format(len(df_plus_six['user'].to_list()), len(df_plus_one['user'].to_list())))

if (len(df_plus_six['user'].to_list()) > 0) & (len(df_plus_one['user'].to_list()) > 0):
    print(1)
    prompt = """
        You run a game called Lucky Guess where users guess the outcome of a game and the score.
        Users who guess it exactly right get 6 points while all other users will get 1 point for guessing within 1 run of the score on either side.
        With this informaiton in mind I would like for you to generate me a short congratulation blurb.
        Some key points regarding the blurb are that the users are all fans of the Orioles and the sport in question is Baseball.
        The blurb should be baseball themed and potentially even include some Orioles facts but don't just talk about Cal Ripken. 
        Please congratulate the following users that guessed exactly {} and the other users are {}. I only need the body of the text nothing else
        and be sure to mention the users.
        """.format(','.join(df_plus_six['user'].to_list()), ','.join(df_plus_one['user'].to_list()))

elif (len(df_plus_six['user'].to_list()) > 0) & (len(df_plus_one['user'].to_list()) < 1):
    print(2)
    prompt = """
        You run a game called Lucky Guess where users guess the outcome of a game and the score.
        Users who guess it exactly right get 6 points while all other users will get 1 point for guessing within 1 run of the score on either side.
        With this informaiton in mind I would like for you to generate me a short congratulation blurb.
        Some key points regarding the blurb are that the users are all fans of the Orioles and the sport in question is Baseball.
        The blurb should be baseball themed and potentially even include some Orioles facts but don't just talk about Cal Ripken. 
        Please congratulate the following users that guessed exactly {}. I only need the body of the text nothing else and be sure to mention the users.
        """.format(','.join(df_plus_six['user'].to_list()))

elif (len(df_plus_six['user'].to_list()) < 1) & (len(df_plus_one['user'].to_list()) > 0):
    print(3)
    prompt = """
        You run a game called Lucky Guess where users guess the outcome of a game and the score.
        Users who guess it exactly right get 6 points while all other users will get 1 point for guessing within 1 run of the score on either side.
        With this informaiton in mind I would like for you to generate me a short congratulation blurb.
        Some key points regarding the blurb are that the users are all fans of the Orioles and the sport in question is Baseball.
        The blurb should be baseball themed and potentially even include some Orioles facts but don't just talk about Cal Ripken. 
        Please congratulate the following users {}. These users did get a point.
        I only need the body of the text nothing else and be sure to mention the users.
        """.format(','.join(df_plus_one['user'].to_list()))

elif(postponed):
    print(4)
    prompt = """
        You run a game called Lucky Guess where users guess the outcome of a game and the score.
        Users who guess it exactly right get 6 points while all other users will get 1 point for guessing within 1 run of the score on either side.
        With this informaiton in mind I would like for you to generate me a short congratulation blurb.
        Some key points regarding the blurb are that the users are all fans of the Orioles and the sport in question is Baseball.
        Unfortunately the game was postponed so just give me a sorriful response and the fun fact about weather. I only need the body of the text nothing else
        """

else:
    print(5)
    prompt = """
        You run a game called Lucky Guess where users guess the outcome of a game and the score.
        Users who guess it exactly right get 6 points while all other users will get 1 point for guessing within 1 run of the score on either side.
        With this informaiton in mind I would like for you to generate me a short congratulation blurb.
        Some key points regarding the blurb are that the users are all fans of the Orioles and the sport in question is Baseball.
        Unfortunately no one scored any points this game so just give me a sorriful response and the fun fact. I only need the body of the text nothing else
        """
congrats_text = generate_output(prompt)
#print(congrats_text)

Number of Exact: 0
Number of +/-1: 0

5


## Publish the entries to Google Sheet

In [19]:
if publish_entries:
    SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

    creds = None
    
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
          creds.refresh(Request())
        else:
          flow = InstalledAppFlow.from_client_secrets_file(
              "00_google_app_oauth.json", SCOPES
          )
          creds = flow.run_local_server(port=0)
    
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

In [20]:
if publish_entries:
    append_values(
          SPREADSHEET_ID,
          "data!A1",
          "USER_ENTERED",
          entries,
    )
else:
    print("entries not published!")

An error occurred: <HttpError 503 when requesting https://sheets.googleapis.com/v4/spreadsheets/1TxZmOdTsMRsLlpJQ5Bu04e3-kWuOdRvK40L4eRYYmWU/values/data%21A1:append?valueInputOption=USER_ENTERED&alt=json returned "The service is currently unavailable.". Details: "The service is currently unavailable.">


## Pull the data for the pitchers

In [21]:
import requests
import urllib3 as ul2

#import mongo_conn

from collections import defaultdict
from bs4 import BeautifulSoup as bs

In [22]:
urls = []
data_loc = 'data/lucky_guess/'

#Get pitchers that are expected to pitch from the mlb website
#prob_pitcher_url = 'https://www.mlb.com/probable-pitchers/'
#prob_pitcher_url = 'https://www.mlb.com/probable-pitchers/2025-03-07'
prob_pitcher_url = 'https://www.mlb.com/probable-pitchers/{}-{}-{}'.format(year, month, day)

#save that file to disk
prob_pitcher_file = data_loc + 'probable_pitchers.html'
with open(prob_pitcher_file, 'wb') as f:
        r = requests.get(prob_pitcher_url)
        f.write(r.text.encode('utf-8'))

soup = bs(open(prob_pitcher_file), 'html.parser')

#find all the pictching match ups
games = soup.findAll('div',{'class':'probable-pitchers__matchup'})

#find specifically the orioles pitchers
teams = []
pitchers = []
game_pitchers = []
urls = []
game_time = ''

for x in games:
    if target_team in x.find('div').text: #found our game
        x.find('div', {'class':'probable-pitchers__team-names'}).findAll('span')

        #Populate the teams array to identify Home and Away
        teams_code = x.find('div', {'class':'probable-pitchers__team-names'}).findAll('span')
        for team in teams_code:
            print(team.text.strip())
            teams.append(team.text.strip())

        pitchers_code = x.findAll('div', {'class':'probable-pitchers__pitcher-name'})
        for pitcher in pitchers_code:
            #print(pitcher.text.strip())
            try:
                print("["+ pitcher.text.strip() + "](https://mlb.com" + pitcher.find('a', href=True)['href'] + ")")
                game_pitchers.append("["+ pitcher.text.strip() + "](https://mlb.com" + pitcher.find('a', href=True)['href'] + ")")
                urls.append("https://mlb.com" + pitcher.find('a', href=True)['href'])
            except:
                print(" ")
                display(HTML("&emsp;Likely no printer listed check " + make_clickable(prob_pitcher_url, "Link")))
                game_pitchers.append("[TBD](" + prob_pitcher_url)
                urls.append(prob_pitcher_url)

        game_time = x.find('div', {'class':'probable-pitchers__game-date-time'}).text.strip()
        print(" ")
        print(game_time)

Red Sox
@
Orioles
[Brayan Bello](https://mlb.com/player/brayan-bello-678394)
[Dietrich Enns](https://mlb.com/player/dietrich-enns-608650)
 
Wed, Aug 27 • 6:35 PM EDT


C:\Users\pjwar\AppData\Local\Temp\ipykernel_104908\1393654273.py:18: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  games = soup.findAll('div',{'class':'probable-pitchers__matchup'})
C:\Users\pjwar\AppData\Local\Temp\ipykernel_104908\1393654273.py:29: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  x.find('div', {'class':'probable-pitchers__team-names'}).findAll('span')
C:\Users\pjwar\AppData\Local\Temp\ipykernel_104908\1393654273.py:32: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  teams_code = x.find('div', {'class':'probable-pitchers__team-names'}).findAll('span')
C:\Users\pjwar\AppData\Local\Temp\ipykernel_104908\1393654273.py:37: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  pitchers_code = x.findAll('div', {'cla

In [23]:
#actually DL the pages to parse the pitcher pages...this could fail if the page is stupid or the pitcher is not usually a starter
for url in urls:
    try:
        url_file = data_loc + url.split('-')[-1] + '.html'
        
        #TODO create the location if it does not exist
        with open(url_file, 'wb') as f:
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
            r = requests.get(url, headers=headers)
            f.write(r.text.encode('utf-8'))
    except:
        print(url)
        print('Failed downloading the page please see the attached url to see why')

In [24]:
#Parse the actual page for the data we want
for url in urls:
    try:
        soup = bs(open(data_loc + url.split('-')[-1] + '.html', 'r', encoding='utf-8'), 'html.parser')
    except:
        name = url.split('-')[1] + ' ' + url.split('-')[2].split('&')[0]
        print('|[' + name + '](' + url + ')|')
        
    try:
        name = soup.find('span', {'class':'player-header--vitals-name'}).text
    
        convert_throws = {'R':'Right', 'L':'Left'}
        throws = soup.find('div', {'class':'player-header--vitals'}).findNext('ul').findNext('li').findNext('li').text
        throws = convert_throws[throws.split(' ')[-1].split('/')[-1]]
    except:
        name = "TBD"
        throws = "None"
        

    # If the parser cant find pitcher stats...just put 0s
    try:
        stats = soup.find('div', {'class':'player-stats-summary-large'}).findNext('tr').findNext('tr').findAll('td')
    except:
        stats = ['0','0','0','0','0','0','0','0','0']

    try:
        print('|'.join([str('[' + name + ']') + str('('  + url + ')'),
                    throws,
                    str(stats[1].text).strip() + '-' + str(stats[2].text.strip()),
                    stats[7].text.strip(),
                    stats[9].text.strip(), 
                    '--', 
                    stats[3].text.strip()]) + '|' +'\n')
        pitchers.append('|'.join([str('[' + name + ']') + str('('  + url + ')'),
                    throws,
                    str(stats[1].text).strip() + '-' + str(stats[2].text.strip()),
                    stats[7].text.strip(),
                    stats[9].text.strip(), 
                    '--', 
                    stats[3].text.strip()]) + '|' +'\n')
    except:
        print('|'.join([str('[' + name + ']') + str('('  + url + ')'),
                    throws,
                    str('NA') + '-' + str('NA'),
                    'NA',
                    'NA', 
                    '--', 
                    'NA']) + '|' +'\n')
        pitchers.append('|'.join([str('[' + name + ']') + str('('  + url + ')'),
                    throws,
                    str('NA') + '-' + str('NA'),
                    'NA',
                    'NA', 
                    '--', 
                    'NA']) + '|' +'\n')

C:\Users\pjwar\AppData\Local\Temp\ipykernel_104908\1287895955.py:13: DeprecationWarning: Call to deprecated method findNext. (Replaced by find_next) -- Deprecated since version 4.0.0.
  throws = soup.find('div', {'class':'player-header--vitals'}).findNext('ul').findNext('li').findNext('li').text
C:\Users\pjwar\AppData\Local\Temp\ipykernel_104908\1287895955.py:22: DeprecationWarning: Call to deprecated method findNext. (Replaced by find_next) -- Deprecated since version 4.0.0.
  stats = soup.find('div', {'class':'player-stats-summary-large'}).findNext('tr').findNext('tr').findAll('td')
C:\Users\pjwar\AppData\Local\Temp\ipykernel_104908\1287895955.py:22: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  stats = soup.find('div', {'class':'player-stats-summary-large'}).findNext('tr').findNext('tr').findAll('td')


[Brayan Bello](https://mlb.com/player/brayan-bello-678394)|Right|10-6|135.0|1.22|--|3.07|

[Dietrich Enns](https://mlb.com/player/dietrich-enns-608650)|Left|2-0|.500|1.000|--|1|



## This is just to auto populate the template as much as possible

In [25]:
template_dict = {
    'away_team' : teams[0],
    'away_selections':', '.join(team_lookup.team_selection[teams[0]]),
    'home_team' : teams[2],
    'home_selections':', '.join(team_lookup.team_selection[teams[2]]),
    'away_pitcher' : pitchers[0],
    'home_pitcher' : pitchers[1],
    'tie_breaker': random.choice(tie_breakers.tie_breakers),
    'away_subreddit': team_lookup.team_subs[teams[0]],
    'home_subreddit': team_lookup.team_subs[teams[2]],
    'game_time': game_time,
    'score_board': scoreboard,
    'congrats_text': congrats_text
}

## Creates hard copy of the post text in case the link fails

In [26]:
in_file = 'reddit_post_template.txt'
out_file = 'output.txt'
 
with open(in_file, 'r') as ifile, open(out_file, 'w') as ofile:
    for line in ifile.readlines():       
        ofile.write(line.format(**template_dict))

## Creates a link that will auto populate the post

In [27]:
game_text = ""
try:
    with open(out_file, 'r') as file:
        game_text = file.read()
        game_text = quote(game_text)
except FileNotFoundError:
    print(f"Error: File not found at path: {out_file}")
except Exception as e:
    print(f"An error occurred: {e}")

title = "[Lucky Guess] {}/{} :: Regular Season Edition".format(month,day)

In [28]:
display(
    HTML("Click to create the post " + 
         make_clickable("https://old.reddit.com/r/orioles/submit?selftext=true&title={}&text={}".format(title, game_text), "Link")))

In [29]:
print('https://old.reddit.com' + submission.permalink)

https://old.reddit.com/r/orioles/comments/1n0a5zg/lucky_guess_0826_regular_season_edition/
